In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
import nltk
from nltk.wsd import lesk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from tensorflow.keras.callbacks import EarlyStopping
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
import keras
from keras import layers
from keras.models import Sequential
rom keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
from keras.metrics import AUC
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import keras_tuner as kt
from sklearn.metrics import classification_report
from kerastuner import HyperParameters as hp
import pickle
import regex as re

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [54]:
data = pd.read_csv('incomplete_annotations_data.csv')
data = data.dropna(subset=['Polarity'])
data

,Brand,Search Term,Comment,Source,Metadata,Subjectivity,Polarity
1,Nike,waste,Kirby would have been a waste of time - why ev...,Twitter,"{'Name': 'LisaKingWheless', 'Handle': '@Lisapc...",1.0,0.0
2,Nike,waste,I wouldn’t spend another dollar at that theate...,Twitter,"{'Name': 'Sheila McSheilerton', 'Handle': '@sh...",1.0,0.0
3,Nike,waste,Call them back and tell them they’re lying bec...,Twitter,"{'Name': 'UncleChrissy', 'Handle': '@uncle_chr...",1.0,0.0
4,Nike,waste,I’m really sitting here going in on myself..li...,Twitter,"{'Name': 'Jade ☥', 'Handle': '@jmerarity', 'Ti...",1.0,1.0
5,Nike,waste,very disappointed in my new Nike Blazer Vint...,Twitter,"{'Name': 'Sailguy', 'Handle': '@NhSailguy', 'T...",1.0,0.0
...,...,...,...,...,...,...,...
2676,Charles & Keith,Charles & Keith,And much more original ang classic right?!!!!,Instagram,{'Likes_and_timestamp': '0 likes on 2024-03-15...,1.0,1.0
2677,Balenciaga,Balenciaga,PP krit Balenciaga ambassador 🔥🔥,Instagram,{'Likes_and_timestamp': '0 likes on 2024-03-05...,1.0,1.0
2678,Louis Vuitton,Louis Vuitton,❤️❤️❤️,Instagram,{'Likes_and_timestamp': '0 likes on 2024-01-17...,1.0,1.0
2679,Tory Burch,Tory Burch,The pale pink in the 6th look is EVERYTHINGGGG...,Instagram,{'Likes_and_timestamp': '0 likes on 2023-09-16...,1.0,1.0


In [55]:
with open('abbreviations_list.pkl', 'rb') as file:
    abbreviations = pickle.load(file)

print(abbreviations)

# Creating extra column for preprocessed text
data['Preprocessed Comment'] = data['Comment']

# Normalizing emojis
!pip install emoji

import emoji

def demojize_with_delimiters(text):
    return emoji.demojize(text, delimiters=(" ", " "))

data['Preprocessed Comment'] = data['Preprocessed Comment'].apply(lambda x: demojize_with_delimiters(x) if isinstance(x, str) else x)

# Lowercasing

data['Preprocessed Comment'] = data['Preprocessed Comment'].apply(lambda x: x.lower() if isinstance(x, str) else x)

# Removing stopwords
nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
    # Ensure the input is a string
    if isinstance(text, str):
        # Tokenize the text into words
        words = nltk.word_tokenize(text)

        # Get the list of stopwords
        stop_words = set(stopwords.words('english'))

        # Remove stopwords from the tokenized words
        filtered_words = [word for word in words if word.lower() not in stop_words]

        # Join the filtered words back into a single string
        filtered_text = ' '.join(filtered_words)

        return filtered_text
    else:
        return text

data['Preprocessed Comment'] = data['Preprocessed Comment'].apply(remove_stopwords)

# Function to manually tokenize text including punctuations
def custom_tokenize(text):
    # Regex pattern to match words (including contractions) and separate punctuation
    tokens = re.findall(r"[\w']+|[.,!?;]", text)
    return tokens

# Normalize slangs and abbreviations
def normalize_slangs_abbreviations_custom(text, slang_dict):
    if isinstance(text, str):
        tokens = custom_tokenize(text)
        normalized_tokens = [slang_dict.get(token.lower(), token) for token in tokens]
        # Reconstruct the text
        normalized_text = ' '.join(normalized_tokens).replace(" ,", ",").replace(" .", ".").replace(" !", "!").replace(" ?", "?")
        return normalized_text
    else:
        return text

data['Preprocessed Comment'] = data['Preprocessed Comment'].apply(lambda x: normalize_slangs_abbreviations_custom(x, abbreviations))

{"ain't": 'is not', "aren't": 'are not', "can't": 'cannot', "can't've": 'cannot have', "'cause": 'because', "could've": 'could have', "couldn't": 'could not', "couldn't've": 'could not have', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hadn't've": 'had not have', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'd've": 'he would have', "he'll": 'he will', "he'll've": 'he he will have', "he's": 'he is', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "I'd": 'I would', "I'd've": 'I would have', "I'll": 'I will', "I'll've": 'I will have', "I'm": 'I am', "I've": 'I have', "i'd": 'i would', "i'd've": 'i would have', "i'll": 'i will', "i'll've": 'i will have', "i'm": 'i am', "i've": 'i have', "isn't": 'is not', "it'd": 'it would', "it'd've": 'it would have', "it'll": 'it will', "it'll've": 'it will have', "it's": 'it is', "let's": 'let us', "ma'am": 'madam', "mayn't": 'may not', "might've": 'migh

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


function for wsd:

In [56]:
stop_words = set(stopwords.words('english'))

def word_sense_disambiguation(sentence):
    #sentence = expand_contractions(sentence, contractions_dict)
    sentence = sentence.lower()
    tokens = word_tokenize(sentence)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    disambiguated_tokens = []

    for token in tokens:
    #     if len(token)==1:
    #         continue
    #     synsets = wn.synsets(token)
    #     if synsets and isinstance(synsets[0], str) and '_' in synsets[0]:
    #         del synsets[0]
    #     #print(token,synsets)
    #     if synsets:
    #         best_synset = max(synsets, key=lambda syn: wn.path_similarity(syn, synsets[0]))
    #         disambiguated_word = best_synset.lemmas()[0].name()
    #         disambiguated_tokens.append(disambiguated_word)
    #     else:
    #         disambiguated_tokens.append(token)
          disambiguated_tokens.append(token)

    return ' '.join(disambiguated_tokens)

In [4]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

--2024-04-09 11:57:09--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.14MB/s    in 2m 42s  

2024-04-09 11:59:51 (5.08 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [5]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [57]:
words = dict()

def add_to_dict(d, filename):
  with open(filename, 'r') as f:
    for line in f.readlines():
      line = line.split(' ')

      try:
        d[line[0]] = np.array(line[1:], dtype=float)
      except:
        continue

add_to_dict(words, 'glove.6B.50d.txt')
#words

In [58]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

In [59]:
lemmatizer = WordNetLemmatizer()

def message_to_token_list(s):
  tokens = tokenizer.tokenize(s)
  lowercased_tokens = [t.lower() for t in tokens]
  lemmatized_tokens = [lemmatizer.lemmatize(t) for t in lowercased_tokens]
  useful_tokens = [t for t in lemmatized_tokens if t in words]
  return useful_tokens


In [60]:
def message_to_word_vectors(message, word_dict=words):
  processed_list_of_tokens = message_to_token_list(message)

  vectors = []

  for token in processed_list_of_tokens:
    if token not in word_dict:
      continue

    token_vector = word_dict[token]
    vectors.append(token_vector)

  return np.array(vectors, dtype=float)

In [61]:
from copy import deepcopy

def pad_X(X, desired_sequence_length=5710):
  X_copy = deepcopy(X)

  for i, x in enumerate(X):
    x_seq_len = x.shape[0]
    sequence_length_difference = desired_sequence_length - x_seq_len

    pad = np.zeros(shape=(sequence_length_difference, 50))

    X_copy[i] = np.concatenate([x, pad])

  return np.array(X_copy).astype(float)

In [62]:
def new_df_to_X_y(dff):
  y = dff['Polarity'].to_numpy().astype(int)

  all_word_vector_sequences = []

  for message in dff['Preprocessed Comment']:
    message_as_vector_seq = word_sense_disambiguation(str(message))
    message_as_vector_seq = message_to_word_vectors(str(message_as_vector_seq))
    if message_as_vector_seq.shape[0] == 0:
      message_as_vector_seq = np.zeros(shape=(1, 50))

    all_word_vector_sequences.append(message_as_vector_seq)

  return all_word_vector_sequences, y

In [63]:
train_df = data.sample(frac=1, random_state=1)
train_df.reset_index(drop=True, inplace=True)

split_index_1 = int(len(train_df) * 0.7)
split_index_2 = int(len(train_df) * 0.85)

train_df, val_df, test_df = train_df[:split_index_1], train_df[split_index_1:split_index_2], train_df[split_index_2:]

len(train_df), len(val_df), len(test_df)

(1092, 234, 234)

In [64]:
X_train = train_df.drop(columns=['Polarity'])
y_train = train_df['Polarity']
oversampler = RandomOverSampler(random_state=42)
X_train_upsampled, y_train_upsampled = oversampler.fit_resample(X_train, y_train)
train_df = pd.concat([X_train_upsampled, y_train_upsampled], axis=1)
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df['Polarity'].value_counts()

Polarity
0.0    603
1.0    603
Name: count, dtype: int64

In [65]:
X_train, y_train = new_df_to_X_y(train_df)
X_train = pad_X(X_train)
X_val, y_val = new_df_to_X_y(val_df)
X_val = pad_X(X_val)
X_test, y_test = new_df_to_X_y(test_df)
X_test = pad_X(X_test)

# LSTM Model

In [66]:
model = Sequential([])

model.add(layers.Input(shape=(5710, 50)))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

In [67]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 5710, 64)          29440     
                                                                 
 dropout_3 (Dropout)         (None, 5710, 64)          0         
                                                                 
 lstm_4 (LSTM)               (None, 5710, 64)          33024     
                                                                 
 dropout_4 (Dropout)         (None, 5710, 64)          0         
                                                                 
 lstm_5 (LSTM)               (None, 5710, 64)          33024     
                                                                 
 dropout_5 (Dropout)         (None, 5710, 64)          0         
                                                                 
 flatten_1 (Flatten)         (None, 365440)           

In [68]:
cp = ModelCheckpoint('model/', save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss=BinaryCrossentropy(),
              metrics=['accuracy', AUC(name='auc')])

In [69]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [70]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, callbacks=[cp,early_stopping])

Epoch 1/20
38/38 [==============================] - 34s 782ms/step - loss: 0.6766 - accuracy: 0.5522 - auc: 0.5830 - val_loss: 0.6862 - val_accuracy: 0.4872 - val_auc: 0.6543
Epoch 2/20
38/38 [==============================] - 19s 492ms/step - loss: 0.6580 - accuracy: 0.5779 - auc: 0.6477 - val_loss: 0.6995 - val_accuracy: 0.6368 - val_auc: 0.6671
Epoch 3/20
38/38 [==============================] - 19s 492ms/step - loss: 0.6487 - accuracy: 0.6161 - auc: 0.6680 - val_loss: 0.6935 - val_accuracy: 0.6453 - val_auc: 0.6921
Epoch 4/20
38/38 [==============================] - 19s 494ms/step - loss: 0.6269 - accuracy: 0.6575 - auc: 0.7059 - val_loss: 0.7033 - val_accuracy: 0.6368 - val_auc: 0.7178
Epoch 5/20
38/38 [==============================] - 19s 499ms/step - loss: 0.6053 - accuracy: 0.6559 - auc: 0.7238 - val_loss: 0.6867 - val_accuracy: 0.6581 - val_auc: 0.7383
Epoch 6/20
38/38 [==============================] - 29s 779ms/step - loss: 0.5786 - accuracy: 0.7056 - auc: 0.7633 - val_loss

In [71]:
best_model = load_model('model/')

In [72]:
test_predictions = (best_model.predict(X_test) > 0.5).astype(int)

print(classification_report(y_test, test_predictions))

8/8 [==============================] - 3s 190ms/step
              precision    recall  f1-score   support

           0       0.60      0.59      0.60        91
           1       0.74      0.75      0.75       143

    accuracy                           0.69       234
   macro avg       0.67      0.67      0.67       234
weighted avg       0.69      0.69      0.69       234



# Keras Tuner (without wsd)

In [73]:
def model_builder(hp):
  model1 = Sequential([])

  model1.add(layers.Input(shape=(5710, 50)))
  model1.add(layers.LSTM(hp.Int('layer_1', min_value=16, max_value=256, step=32), return_sequences=True))
  model1.add(layers.Dropout(hp.Float('dropout_rate_1', min_value=0.0, max_value=0.5, step=0.1)))
  model1.add(layers.LSTM(hp.Int('layer_2', min_value=16, max_value=256, step=32), return_sequences=True))
  model1.add(layers.Dropout(hp.Float('dropout_rate_2', min_value=0.0, max_value=0.5, step=0.1)))
  model1.add(layers.LSTM(hp.Int('layer_3', min_value=16, max_value=256, step=32), return_sequences=True))
  model1.add(layers.Dropout(hp.Float('dropout_rate_3', min_value=0.0, max_value=0.5, step=0.1)))
  model1.add(layers.Flatten())
  model1.add(layers.Dense(1, activation='sigmoid'))

  model1.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss=BinaryCrossentropy(),
                metrics=['accuracy', AUC(name='auc')])

  return model1

In [79]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='dir',
                     project_name='x',
                     )

In [80]:
# import shutil

# shutil.rmtree('/content/dir')

In [81]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [82]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[stop_early])

Trial 30 Complete [00h 03m 56s]
val_accuracy: 0.6111111044883728

Best val_accuracy So Far: 0.6282051205635071
Total elapsed time: 00h 51m 49s


In [83]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [84]:
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val),
                    callbacks=[stop_early])

Epoch 1/50
38/38 [==============================] - 28s 625ms/step - loss: 0.6813 - accuracy: 0.5813 - auc: 0.6156 - val_loss: 0.6832 - val_accuracy: 0.4915 - val_auc: 0.6543
Epoch 2/50
38/38 [==============================] - 22s 580ms/step - loss: 0.6688 - accuracy: 0.5622 - auc: 0.6348 - val_loss: 0.6793 - val_accuracy: 0.4915 - val_auc: 0.6513
Epoch 3/50
38/38 [==============================] - 22s 575ms/step - loss: 0.6631 - accuracy: 0.5887 - auc: 0.6585 - val_loss: 0.6787 - val_accuracy: 0.5214 - val_auc: 0.6528
Epoch 4/50
38/38 [==============================] - 22s 571ms/step - loss: 0.6587 - accuracy: 0.5929 - auc: 0.6620 - val_loss: 0.6764 - val_accuracy: 0.5726 - val_auc: 0.6561
Epoch 5/50
38/38 [==============================] - 21s 566ms/step - loss: 0.6554 - accuracy: 0.6169 - auc: 0.6598 - val_loss: 0.6717 - val_accuracy: 0.6154 - val_auc: 0.6572
Epoch 6/50
38/38 [==============================] - 22s 572ms/step - loss: 0.6522 - accuracy: 0.6285 - auc: 0.6566 - val_loss

In [85]:
history.history

{'loss': [0.6812644004821777,
  0.6688125729560852,
  0.6631354093551636,
  0.6587415337562561,
  0.655370831489563,
  0.6522135138511658,
  0.6490049362182617,
  0.6478546261787415,
  0.6431553363800049,
  0.6414837837219238,
  0.6376879811286926,
  0.6326668858528137,
  0.6297457218170166,
  0.6274489760398865,
  0.6233307123184204,
  0.619789719581604,
  0.613732635974884,
  0.6107355356216431,
  0.6016005873680115,
  0.5969323515892029,
  0.5884407758712769,
  0.5822885632514954,
  0.5781131982803345,
  0.5749149918556213,
  0.5673708915710449,
  0.5650833249092102,
  0.5617639422416687,
  0.5527589917182922,
  0.5538033246994019,
  0.5474143624305725,
  0.5416439175605774,
  0.545857310295105,
  0.5395934581756592,
  0.5360227823257446,
  0.5462039113044739,
  0.5314695835113525,
  0.5337784886360168,
  0.5266908407211304,
  0.5264688730239868],
 'accuracy': [0.5812603831291199,
  0.5621890425682068,
  0.5887230634689331,
  0.592868983745575,
  0.6169154047966003,
  0.628524065017

In [86]:
test_predictions = (best_model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, test_predictions))

8/8 [==============================] - 3s 268ms/step
              precision    recall  f1-score   support

           0       0.61      0.71      0.66        91
           1       0.80      0.71      0.75       143

    accuracy                           0.71       234
   macro avg       0.71      0.71      0.71       234
weighted avg       0.73      0.71      0.72       234



In [88]:
best_model.save('LSTM_best_model_polarity.h5')